# Linear Solver Performance: Evaluation of MPI Parallel Benchmarks

In [ ]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

In [ ]:
#r "BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net6.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
string PROJECT_NAME = System.Environment.GetEnvironmentVariable("LinslvPerfPar") ?? "LinslvPerfPar"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

In [ ]:
databases

In [ ]:
wmg.Sessions

In [ ]:
//var SS = wmg.Sessions[14];
//SS

In [ ]:
//SS.Delete(true)

In [ ]:
//SS.KeysAndQueries["Conv"]

In [ ]:
//foreach(var si in wmg.Sessions.Where(Si => Si.Name.Contains("GridCration")))
//    si.Delete(true);

In [ ]:
//wmg.Sessions.Where(Si => !Si.Name.Contains("GridCreation") && !Si.KeysAndQueries.ContainsKey("Conv"))

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => !Si.Name.Contains("GridCreation") &&
                                              (Si.SuccessfulTermination == false
                                              || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions

## Create Table for Post-Processing

The timing information that is requires for this study is not present in the default session table.
Instead, it must be extracted from the *session profiling*.

To obtain timing-measurements, the instrumentation generated by certain `BlockTrace`-blocks 
within the `UniSolver`, resp. the `XdgBDFTimestepping` class is extracted and added to the data table:

In [ ]:
// evaluators to add additional columns to the session table
static class AddCols {
    static int GetSolType(ISessionInfo SI) {
        if(SI.GetControl().GetSolverType().Name.Contains("XNSE"))
            return 3;
        if(SI.GetControl().GetSolverType().Name.Contains("XdgPoisson3Main"))
            return 2;
        return 1;
    }
    
    
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling(0)[0];
        ilPSP.Tracing.MethodCallRecord nd;
        if(GetSolType(SI) == 3)
            nd = mcr.FindChildren("*.XdgTimestepping.ComputeOperatorMatrix").Single();
        else 
            nd  = mcr.FindChildren("MatrixAssembly").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling(0)[0];
        var nd  = mcr.FindChildren("Aggregation_basis_init").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling(0)[0];
        var nd  = mcr.FindChildren("Solver_Init").Single();
        //var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling(0)[0];
        var nd  = mcr.FindChildren("Solver_Run").Single();
        //var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_InitAndRun_time(ISessionInfo SI) {
        double agitime = (double) Aggregation_basis_init_time(SI);
        double initime = (double) Solver_Init_time(SI);
        double runtime = (double) Solver_Run_time(SI);
        
        return agitime + initime + runtime;
    }
    static public object Solver_TimePerIter(ISessionInfo SI) {
        double runtime = (double) Solver_Run_time(SI);
        double NoOfItr = Convert.ToDouble(SI.KeysAndQueries["NoIter"]);
        return runtime/NoOfItr;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
    }
    static public object DOFsPerCore(ISessionInfo SI){
        return  System.Convert.ToDouble(SI.KeysAndQueries["DOFs"])/System.Convert.ToDouble(NoOfCores(SI));
    }
    static public object ComputeNodeName(ISessionInfo SI) {
        return SI.ComputeNodeNames.First();
    }
    static public object SessionNameWoSize(ISessionInfo SI) {
        string Nmn = SI.Name;
        return Nmn.Substring(0,Nmn.IndexOf("Sz")-1);
    }
    
    //static public object DOFs(ISessionInfo SI) {
    //    int NoOfItr = Convert.ToInt32(SI.KeysAndQueries["NoIter"]);
    //    return NoOfItr;
    //}
    
    static public object DGdegree(ISessionInfo SI) {
        switch(GetSolType(SI)) {
            case 3:
            return SI.KeysAndQueries.Single(kv => kv.Key.Contains("DGdegree:Velocity")).Value;
            case 2:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:u").Value;    
            case 1:
            return SI.KeysAndQueries.Single(kv => kv.Key == "DGdegree:T").Value;
        }
        throw new ArgumentException();
    }
}

In [ ]:
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("Solver_InitAndRun_time", AddCols.Solver_InitAndRun_time);
wmg.AdditionalSessionTableColums.Add("Solver_TimePerIter", AddCols.Solver_TimePerIter);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);
wmg.AdditionalSessionTableColums.Add("ComputeNodeName", AddCols.ComputeNodeName);
wmg.AdditionalSessionTableColums.Add("DGdegree", AddCols.DGdegree);
wmg.AdditionalSessionTableColums.Add("DOFsPerCore", AddCols.DOFsPerCore);
wmg.AdditionalSessionTableColums.Add("SessionNameWoSize", AddCols.SessionNameWoSize);

In [ ]:
var SessTab = wmg.SessionTable;

List of all available Data Columns:

In [ ]:
//SessTab.GetColumnNames().ToConcatString("", "; ", "")

In [ ]:
var SubTab = SessTab.ExtractColumns(
    "SessionName", "SessionNameWoSize", "DGdegree", "Grid:NoOfCells", "LinearSolver.Name", "LinearSolver.Shortname", "DOFs", "DOFsPerCore", "MatrixAssembly",
    "Grid:SpatialDimension", "NoOfCores",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "Solver_InitAndRun_time", "NoIter", 
    "Solver_TimePerIter", "ComputeNodeName", "RegularTerminated");

In [ ]:
// remove all rows from "Dummy" sessions for grid creation
SubTab = SubTab.ExtractRows(delegate(int iRow, IDictionary<string, object> row) {
    return !((string)row["SessionName"]).Contains("GridCreation");
});

In [ ]:
// Filename
var now         = DateTime.Now;
string docName  = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
SubTab.SaveToFile(docName + ".json");
SubTab.ToCSVFile(docName + ".csv");

Only consider runs which have been successful:

In [ ]:
SubTab = SubTab.ExtractRows(delegate(int iRow, IDictionary<string, object> row) {
    return (bool)(row["RegularTerminated"]);
});

## Vizualisation of Results

The following data is available:

In [ ]:
SubTab.GetColumnNames()

Available DG degrees:

In [ ]:
var DGdegrees = SubTab.GetColumn<int>("DGdegree").ToSet().OrderBy(s => s).ToArray();
DGdegrees

Available Grid Resolutions:

In [ ]:
var ResolutionS = SubTab.GetColumn<int>("Grid:NoOfCells").ToSet().OrderBy(s => s).ToArray();
ResolutionS

In [ ]:
var MPIsizeS = SubTab.GetColumn<int>("NoOfCores").ToSet().OrderBy(s => s).ToArray();
MPIsizeS

Cases investigated:

In [ ]:
const string Poisson = "SIP_Poisson";
const string XPoisson = "XdgPoisson";
const string Stokes2D = "BottiPietroStokes2D";
const string Stokes3D = "BottiPietroStokes3D";
const string XStokes = "XdgStokes";
string[] AllCases = new string[] { Poisson, XPoisson, Stokes2D, Stokes3D, XStokes };

### Macro Routine for Multiplot

The following routine combines the plotting code which is common for all sub-plot in this evaluation;
only the y-axis needs to be specified. 

In [ ]:
int MaxMpiSize(int J, int k) {
    int sz = -1;
    int L = SubTab.Rows.Count;
    for (int i = 0; i < L; i++) {
        var orgRow = SubTab.Rows[i];
        int Res = Convert.ToInt32(orgRow["Grid:NoOfCells"]);
        if(Res != J)
            continue;
        int DgDeg = Convert.ToInt32(orgRow["DGdegree"]);
        if(DgDeg != k)
            continue;
        int sz_i = Convert.ToInt32(orgRow["NoOfCores"]);
        sz = Math.Max(sz, sz_i);
    }
    return sz;
}

In [ ]:
PlotFormat SlvCode2Pltfmt(string caseName, int J, int k, int sz_max) { 
    var Fmt = new PlotFormat();

    Fmt.PointType = PointTypes.Diamond;

    //Console.WriteLine("name is: " + solver_name); 
    Fmt.PointSize = 0.85;
    Fmt.LineWidth = 2;    
    Fmt.Style     = Styles.LinesPoints;
    
    if(caseName.Contains("Xdg"))
        Fmt.DashType = DashTypes.Solid;
    else
        Fmt.DashType = DashTypes.Dashed;
        

    Fmt.SetLineColorFromIndex(Array.IndexOf(ResolutionS, J)); 
    
    int i = Array.IndexOf(MPIsizeS, sz_max);
    var values = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
    Fmt.PointType = values[i % values.Length];

    return Fmt;
}

In [ ]:
string ExtractCase(string sessionName) {
    return AllCases.Single(caseName => sessionName.Contains(caseName));
}

In [ ]:
Plot2Ddata[,] PlotSolverBehave(string Yname, bool LogY, double yMin, double yMax, double LegendYpos) {

int rows    = DGdegrees.Length;
int columns = 2;
string[] ignore_solvers = {};
Plot2Ddata[,] multiplots = new Plot2Ddata[rows + 1,columns];
int pDegree = 0;
for(int iRow = 0; iRow < rows; iRow++) {
for(int iCol = 0; iCol < columns; iCol++) {
    
    //if(pDegree > rows*columns-1)
    //    continue;
    //int tmpDG = -1;
    //if(pDegree < DGdegrees.Length)
    //    tmpDG = DGdegrees[pDegree];
    
    int tmpDG = DGdegrees[iRow];
    
    //Create Graphs
    multiplots[iRow,iCol] = SubTab.ToPlot("DOFsPerCore", Yname, // column for x- and y
       delegate (int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k = Convert.ToInt32(Row["DGdegree"]);
           if(k != tmpDG) {
                // degree does not match -> not in this plot
                Nmn = null;
                Fmt = null;
                return;
           }
 
           string solver_name = (string) (Row["LinearSolver.Shortname"]);
           //ignore the solvers specified in ingore_solvers
           foreach(string sc in ignore_solvers){
               if(solver_name == sc){
                   System.Console.WriteLine("skipped");
                   Nmn = null;
                   Fmt = null;
               return;
               }
           }
           
           string caseName = ExtractCase( (string) Row["SessionNameWoSize"]);
           if(iCol == 0) {
               // in Column 0, draw only Poisson
               if(!caseName.Contains("Poisson")) {
                   Nmn = null;
                   Fmt = null;
                   return;
               }
           } else if(iCol == 1) {
               // in Column 1, draw only Poisson
               if(!caseName.Contains("Stokes")) {
                   Nmn = null;
                   Fmt = null;
                   return;
               }
           } else {
               throw new NotImplementedException();
           }
           
           int J = Convert.ToInt32(Row["Grid:NoOfCells"]);
           int sz = Convert.ToInt32(Row["NoOfCores"]);
           int sz_max = MaxMpiSize(J, k);

           //process the other solvers
           Fmt = SlvCode2Pltfmt(caseName, J, k, sz_max);
           //Nmn = (string) Row["SessionNameWoSize"];
           Nmn = $"J{J}-Sz{sz_max}";
       });
    
       // plot the linear behavior reference line
       double[] dof = new[] { 1e3, 1e6 }; // x-limits of the reference-line-plot
       double[] linT = dof.Select(x => x*0.001).ToArray();
       var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
       linP.Format.FromString("- black");
       ArrayTools.AddToArray(linP, ref multiplots[iRow,iCol].dataGroups);
 
       //all about axis
       string Title = string.Format("$k = {0}$", tmpDG);
       multiplots[iRow,iCol].Ylabel = Title;
       multiplots[iRow,iCol].LogX = true;
       multiplots[iRow,iCol].LogY = LogY;
    
       //specify range of axis
       multiplots[iRow,iCol].YrangeMin = yMin;
       multiplots[iRow,iCol].YrangeMax = yMax;
       multiplots[iRow,iCol].XrangeMin = 1e2;
       multiplots[iRow,iCol].XrangeMax = 1e6;
    
       //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
       //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
       //multiplots[iRow,iCol].X2rangeMin = 1e2;
       //multiplots[iRow,iCol].X2rangeMax = 1e7;
    
       //spacing around plots
       multiplots[iRow,iCol].ShowLegend = false;
       multiplots[iRow,iCol].tmargin = 0;
       multiplots[iRow,iCol].bmargin = 2;
       multiplots[iRow,iCol].lmargin = 5;
       multiplots[iRow,iCol].rmargin = 5;
       multiplots[iRow,iCol].ShowXtics = false;

       //I am legend ...
       if(iRow == 0) {
          multiplots[iRow,iCol].ShowLegend = true;
          //multiplots[iRow,iCol].LegendAlignment = new string[]{"o", "r", "t" };
          multiplots[iRow,iCol].LegendFont = 12;
          multiplots[iRow,iCol].Legend_maxrows = 100;
          multiplots[iRow,iCol].LegendPosition = new double[] { 4e3, LegendYpos };
          //multiplots[iRow,iCol].LegendSwap  = true;
       }
       //and i am special ...
       if(iRow == rows - 1)
           multiplots[iRow,iCol].ShowXtics = true;
    pDegree++;
}                        
}
//multiplots.PlotCairolatex().WriteMinimalCompileableExample("latex/solvers.tex");
//multiplots.AddDummyPlotsForLegend(3,0);
return multiplots;
}

In [ ]:
var multiplotsRtime = PlotSolverBehave("Solver_InitAndRun_time", true, 1e-1, 0.5e3, 1e-11);
//multiplots.PlotCairolatex().PlotNow()
//multiplots.AddDummyPlotsForLegend(3,0);
multiplotsRtime.ToGnuplot().PlotSVG(xRes:800,yRes:1200)